# 3rd model to train image classifier
> using Keras and Tensorflow

**Import the libraries and prepare the dataset.**

In [121]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random

print(tf.__version__)

1.11.0


In [122]:
import os 
from tensorflow.keras.preprocessing import image
 
import pathlib
def getImagePaths(rootPath):
  data_root = pathlib.Path('./data/train/')
  image_paths = list(data_root.glob('*/*'))
  image_paths = [str(path) for path in image_paths if not str(path).endswith('DS_Store')]
  random.shuffle(image_paths)
  return image_paths

#image resize -> PIL -> tensorflow 
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize_images(image, [128, 128])
  image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)


In [123]:
# Prepare the train Dataset
all_image_paths = getImagePaths('./data/train/')
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
print(label_names)
label_to_index = dict((name, index) for index,name in enumerate(label_names))
print(label_to_index)

all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
number_of_images = len(all_image_labels)
all_image_array = []
for image_path in all_image_paths:
  all_image_array.append(load_and_preprocess_image(image_path))

all_image_tensor = tf.convert_to_tensor(all_image_array, dtype=tf.float32)
train_dataset_reshape = tf.reshape(all_image_tensor, [number_of_images, 128, 128, 1])
train_dataset_reshape.shape

# Prepare the prediction Dataset
all_predict_paths = getImagePaths('./data/test/')
all_predict_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_predict_paths]
number_of_predict = len(all_predict_labels)
all_predict_array = []
for image_path in all_image_paths:
  all_image_array.append(load_and_preprocess_image(image_path))

all_image_tensor = tf.convert_to_tensor(all_image_array, dtype=tf.float32)
train_dataset_reshape = tf.reshape(all_image_tensor, [number_of_images, 128, 128, 1])
train_dataset_reshape.shape


['barramundi', 'bream', 'snapper']
{'barramundi': 0, 'snapper': 2, 'bream': 1}


TensorShape([Dimension(36), Dimension(128), Dimension(128), Dimension(1)])

In [124]:
# image_path = all_image_paths[1]
# print(image_path)
# x = load_and_preprocess_image(image_path)
# x.shape
# plt.imshow(load_and_preprocess_image(image_path))
# plt.grid(False)
# plt.show()

**Setup the model and train it**

In [125]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128, 1)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

In [126]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [127]:
model.fit(train_dataset_reshape, all_image_labels, epochs=50, steps_per_epoch=1)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 1.7024 - acc: 0.3889
Epoch 2/50
1/1 [==============================] - 0s 13ms/step - loss: 9.9046 - acc: 0.3056
Epoch 3/50
1/1 [==============================] - 0s 12ms/step - loss: 10.2352 - acc: 0.3056
Epoch 4/50
1/1 [==============================] - 0s 17ms/step - loss: 10.6078 - acc: 0.3056
Epoch 5/50
1/1 [==============================] - 0s 14ms/step - loss: 10.5893 - acc: 0.3056
Epoch 6/50
1/1 [==============================] - 0s 11ms/step - loss: 10.4386 - acc: 0.3056
Epoch 7/50
1/1 [==============================] - 0s 16ms/step - loss: 10.2431 - acc: 0.3333
Epoch 8/50
1/1 [==============================] - 0s 16ms/step - loss: 10.0743 - acc: 0.3333
Epoch 9/50
1/1 [==============================] - 0s 17ms/step - loss: 9.7728 - acc: 0.3333
Epoch 10/50
1/1 [==============================] - 0s 15ms/step - loss: 9.0071 - acc: 0.3889
Epoch 11/50
1/1 [==============================] - 0s 12ms/step - loss: 6.5

Test the model and make use of that.

In [128]:
test_loss, test_acc = model.evaluate(train_dataset_reshape, all_image_labels, steps=1)
print('Test accuracy:', test_acc)

1/1 [==============================] - 1s 573ms/step
Test accuracy: 0.6944444179534912


In [129]:
predictions = model.predict(train_dataset_reshape, steps=1)
for item in predictions:
  print(np.argmax(item), end=', ')
print()
print(all_image_labels)

1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 
[1, 2, 2, 1, 0, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 0, 0, 1, 0, 1, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1, 1, 2, 0, 0, 0]
